In [1]:
pip install google-cloud-aiplatform

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.1 MB 631 kB/s eta 0:00:01     |█████████████▍                  | 3.0 MB 669 kB/s eta 0:00:07
     |████████████████████████████████| 384 kB 616 kB/s eta 0:00:01
     |████████████████████████████████| 131 kB 539 kB/s eta 0:00:01
     |████████████████████████████████| 244 kB 543 kB/s eta 0:00:01
     |████████████████████████████████| 50 kB 653 kB/s eta 0:00:01
     |████████████████████████████████| 160 kB 595 kB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 279 kB/s eta 0:00:01
     |████████████████████████████████| 210 kB 342 kB/s eta 0:00:01
     |████████████████████████████████| 431 kB 440 kB/s eta 0:00:01
     |████████████████████████████████| 417 kB 555 kB/s eta 0:00:01
     |████████████████████████████████| 221 kB 618 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 690 kB/s eta 0:00:01
     |████████████████████████████████| 1

In [5]:
pip install gcsfs

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 184 kB 381 kB/s eta 0:00:01
     |████████████████████████████████| 455 kB 483 kB/s eta 0:00:01
     |████████████████████████████████| 52 kB 621 kB/s eta 0:00:01
     |████████████████████████████████| 63 kB 673 kB/s eta 0:00:01
     |████████████████████████████████| 45 kB 739 kB/s eta 0:00:01
     |████████████████████████████████| 92 kB 647 kB/s eta 0:00:01
     |████████████████████████████████| 151 kB 599 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.cloud.aiplatform
print("Google Cloud AI Platform module is available.")

/Users/ruchika/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Google Cloud AI Platform module is available.


In [6]:
import gcsfs
print("gcsfs is installed and working!")

gcsfs is installed and working!


In [ ]:
from google.auth import credentials
from google.auth import default
from google.cloud import storage

# Use the default credentials set by gcloud (it will read from your .config directory)
credentials, project = default()

# Initialize the Google Cloud Storage client with the credentials
client = storage.Client(credentials=credentials, project=project)


/Users/ruchika/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


BadRequest: 400 GET https://storage.googleapis.com/storage/v1/b?projection=noAcl&prettyPrint=false: Required parameter: project

In [2]:
import pandas as pd

# Load dataset from Google Cloud Storage (GCS)
df = pd.read_csv("gs://mlb-prospect-data/cleaned_mlb_homeruns.csv")

# Print column names
print("Columns in dataset:", df.columns.tolist())

Columns in dataset: ['play_id', 'ExitVelocity', 'HitDistance', 'LaunchAngle', 'Year', 'WAR', 'PlayerName']


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14064 entries, 0 to 14063
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   play_id       14064 non-null  object 
 1   ExitVelocity  14064 non-null  float64
 2   HitDistance   14064 non-null  float64
 3   LaunchAngle   14064 non-null  float64
 4   Year          14064 non-null  int64  
 5   WAR           14064 non-null  float64
 6   PlayerName    14064 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 769.2+ KB


In [4]:
from google.cloud import aiplatform
import pandas as pd

# Initialize Vertex AI
aiplatform.init(project="mlb-project-449501", location="us-central1")

# ✅ Load dataset from Google Cloud Storage into Pandas DataFrame
df = pd.read_csv("gs://mlb-prospect-data/cleaned_mlb_homeruns.csv")

# ✅ Print column names to verify target variable exists
print("Columns in dataset:", df.columns.tolist())

# ✅ Register dataset in Vertex AI if not already registered
dataset = aiplatform.TabularDataset.create(
    display_name="mlb_prospect_prediction",
    gcs_source="gs://mlb-prospect-data/cleaned_mlb_homeruns.csv"
)

# ✅ Train AutoML Model
model = aiplatform.AutoMLTabularTrainingJob(
    display_name="prospect_model",
    optimization_prediction_type="regression"  # Change to 'classification' if predicting player tiers
)

# ✅ Run training job
model = model.run(
    dataset=dataset,
    target_column="WAR",  # Ensure "WAR" exists in dataset
    model_display_name="mlb_prospect_model",
    budget_milli_node_hours=5000,  # Increase for better results
    disable_early_stopping=False  # Enables early stopping to prevent overfitting
)

print("🚀 Model training started successfully!")

Columns in dataset: ['play_id', 'ExitVelocity', 'HitDistance', 'LaunchAngle', 'Year', 'WAR', 'PlayerName']
Creating TabularDataset
Create TabularDataset backing LRO: projects/811481050675/locations/us-central1/datasets/8448861202842386432/operations/5854057088921108480
TabularDataset created. Resource name: projects/811481050675/locations/us-central1/datasets/8448861202842386432
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/811481050675/locations/us-central1/datasets/8448861202842386432')
No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
The column transformation of type 'auto' was set for the following columns: ['PlayerName', 'ExitVelocity', 'play_id', 'Year', 'LaunchAngle', 'HitDistance'].
No dataset split provided. The service will use a default split.
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5755940588268552192?project=

RetryError: Timeout of 120.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv4:64.233.176.95:443: tcp handshaker shutdown

In [6]:
from google.cloud import aiplatform
from google.api_core import retry
import pandas as pd

# Initialize Vertex AI
aiplatform.init(project="mlb-project-449501", location="us-central1")

# ✅ Load dataset from Google Cloud Storage into Pandas DataFrame
df = pd.read_csv("gs://mlb-prospect-data/cleaned_mlb_homeruns.csv")

# ✅ Print column names to verify target variable exists
print("Columns in dataset:", df.columns.tolist())

# ✅ Register dataset in Vertex AI if not already registered
dataset = aiplatform.TabularDataset.create(
    display_name="mlb_prospect_prediction",
    gcs_source="gs://mlb-prospect-data/cleaned_mlb_homeruns.csv"
)

# ✅ Train AutoML Model
model = aiplatform.AutoMLTabularTrainingJob(
    display_name="prospect_model",
    optimization_prediction_type="regression"  # Change to 'classification' if predicting player tiers
)

# Define the retry logic using google.api_core.retry
custom_retry = retry.Retry(
    predicate=retry.if_exception_type(Exception),  # Retry on any exception
    deadline=600,  # Timeout after 10 minutes
    maximum=10,    # Retry up to 10 times
)

# ✅ Run training job with retry logic
def run_model_with_retry():
    try:
        model_run = model.run(
            dataset=dataset,
            target_column="WAR",  # Ensure "WAR" exists in dataset
            model_display_name="mlb_prospect_model",
            budget_milli_node_hours=5000,  # Increase for better results
            disable_early_stopping=False  # Enables early stopping to prevent overfitting
        )
        print("🚀 Model training started successfully!")
        return model_run
    except Exception as e:
        print(f"Error while running model: {e}")
        raise

# Apply retry mechanism
custom_retry(run_model_with_retry)()


Columns in dataset: ['play_id', 'ExitVelocity', 'HitDistance', 'LaunchAngle', 'Year', 'WAR', 'PlayerName']
Creating TabularDataset
Create TabularDataset backing LRO: projects/811481050675/locations/us-central1/datasets/5597871532484329472/operations/5577754214907510784
TabularDataset created. Resource name: projects/811481050675/locations/us-central1/datasets/5597871532484329472
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/811481050675/locations/us-central1/datasets/5597871532484329472')
No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
The column transformation of type 'auto' was set for the following columns: ['PlayerName', 'ExitVelocity', 'play_id', 'Year', 'LaunchAngle', 'HitDistance'].
No dataset split provided. The service will use a default split.
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4837909951726747648?project=

resource name: projects/811481050675/locations/us-central1/models/8929948667285078016

In [5]:
datasets = aiplatform.TabularDataset.list()
print([d.resource_name for d in datasets])

['projects/811481050675/locations/us-central1/datasets/4336159949378813952']


In [7]:


aiplatform.init(project="mlb-project-449501", location="us-central1")

endpoint = aiplatform.Endpoint.create(display_name="mlb_prospect_endpoint")
print(f"✅ Endpoint created: {endpoint.resource_name}")


Creating Endpoint
Create Endpoint backing LRO: projects/811481050675/locations/us-central1/endpoints/8487940594872614912/operations/4002299187839369216
Endpoint created. Resource name: projects/811481050675/locations/us-central1/endpoints/8487940594872614912
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/811481050675/locations/us-central1/endpoints/8487940594872614912')
✅ Endpoint created: projects/811481050675/locations/us-central1/endpoints/8487940594872614912


In [10]:
from google.cloud import aiplatform

aiplatform.init(project="mlb-project-449501", location="us-central1")

models = aiplatform.Model.list()
for model in models:
    print(f"Model Display Name: {model.display_name}, Model ID: {model.resource_name}")


Model Display Name: mlb_prospect_model, Model ID: projects/811481050675/locations/us-central1/models/8929948667285078016
Model Display Name: mlb_prospect_model, Model ID: projects/811481050675/locations/us-central1/models/3363781002831855616


In [12]:
from google.cloud import aiplatform

# Initialize Vertex AI
aiplatform.init(project="mlb-project-449501", location="us-central1")

# Load the correct model
model = aiplatform.Model("8929948667285078016")  # Use your correct Model ID

# Load the existing endpoint as an object
endpoint = aiplatform.Endpoint("8487940594872614912")  # Use your correct Endpoint ID

# Deploy the model
deployed_model = model.deploy(
    endpoint=endpoint,
    machine_type="n1-standard-4"  # Change as per your requirements
)

print("🚀 Model deployed successfully!")


Deploying model to Endpoint : projects/811481050675/locations/us-central1/endpoints/8487940594872614912
Deploy Endpoint model backing LRO: projects/811481050675/locations/us-central1/endpoints/8487940594872614912/operations/5812746238042308608
Endpoint model deployed. Resource name: projects/811481050675/locations/us-central1/endpoints/8487940594872614912
🚀 Model deployed successfully!


In [13]:
import pandas as pd

# Load historical (training) data
historical_data_path = "./dataset/cleaned_mlb_homeruns.csv"
df_historical = pd.read_csv(historical_data_path)

# Load live game data
live_data_path = "./dataset/live_game_data.csv"
df_live = pd.read_csv(live_data_path)

# 🏷️ **Ensure Only One `play_id` Per Player in Historical Data (Take First Occurrence)**
df_historical_unique = df_historical.groupby("PlayerName", as_index=False).first()  # Keep only first occurrence

# 🏷️ **Match Player Names to Retrieve `play_id` from Historical Data**
df_live = df_live.merge(df_historical_unique[["PlayerName", "play_id"]], on="PlayerName", how="left")

# 🔄 **Replace `PlayerID` with `play_id` from historical data**
df_live.drop(columns=["PlayerID"], inplace=True)  # Remove old PlayerID column
df_live.rename(columns={"play_id": "PlayerID"}, inplace=True)  # Rename play_id to PlayerID

# Check for missing player IDs
missing_players = df_live[df_live["PlayerID"].isna()]
if not missing_players.empty:
    print(f"⚠️ Warning: {len(missing_players)} players were not found in historical data.")
    df_live.loc[df_live["PlayerID"].isna(), "PlayerID"] = range(100000, 100000 + len(missing_players))  # Assign unique IDs

# Save updated live game data
df_live.to_csv("./dataset/updated_live_game_data.csv", index=False)
print("✅ Updated live game data saved as 'updated_live_game_data.csv'")


⚠️ Warning: 4 players were not found in historical data.
✅ Updated live game data saved as 'updated_live_game_data.csv'


In [2]:
import pandas as pd

# Load updated live game data (with correct PlayerID)
file_path = "./dataset/updated_live_game_data.csv"
df_live = pd.read_csv(file_path)

# 🛠️ **Rename columns to match training dataset format**
df_live.rename(columns={
    "OPS": "ExitVelocity",       # Mapping OPS to ExitVelocity
    "HomeRuns": "HitDistance",   # Mapping HomeRuns to HitDistance
    "Strikeouts": "LaunchAngle"  # Mapping Strikeouts to LaunchAngle
}, inplace=True)

# 🔄 **Apply Transformations to Match Training Data**
df_live["ExitVelocity"] *= 100       # Assuming OPS is correlated to ExitVelocity
df_live["HitDistance"] *= 30         # Assuming each HR is ~30 feet in distance
df_live["LaunchAngle"] *= 2          # Assuming each strikeout influences launch angle

# Add "Year" column for model consistency
df_live["Year"] = 2025  # Set to current year

# **Keep only the exact columns used in training**
df_cleaned = df_live[["PlayerID", "ExitVelocity", "HitDistance", "LaunchAngle", "Year", "PlayerName"]]

# Save cleaned data for debugging
df_cleaned.to_csv("final_transformed_live_game_data.csv", index=False)
print("✅ Transformed live game data saved as 'final_transformed_live_game_data.csv'")


✅ Transformed live game data saved as 'final_transformed_live_game_data.csv'


In [3]:

# 🏷️ **Rename `PlayerID` to `play_id` to match the trained model's input**
df_cleaned.rename(columns={"PlayerID": "play_id"}, inplace=True)

# 🔄 **Convert all numerical values to strings (Vertex AI expects strings)**
for col in ["ExitVelocity", "HitDistance", "LaunchAngle", "Year"]:
    df_cleaned[col] = df_cleaned[col].astype(str)

# Convert DataFrame to prediction format (drop non-feature columns)
instances = df_cleaned.to_dict(orient="records")

# 🔮 **Predict WAR**
response = endpoint.predict(instances)

# **Attach Predictions to DataFrame**
df_cleaned["Predicted_WAR"] = response.predictions

# Print the results
print(df_cleaned[["PlayerName", "Predicted_WAR"]])

# **Save predictions to CSV**
df_cleaned.to_csv("./dataset/predicted_live_game_data.csv", index=False)
print("✅ Predictions saved to 'predicted_live_game_data.csv'")


/var/folders/_d/pn647pfx16x6j369mlcqzzn40000gn/T/ipykernel_27942/4026449429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"PlayerID": "play_id"}, inplace=True)
/var/folders/_d/pn647pfx16x6j369mlcqzzn40000gn/T/ipykernel_27942/4026449429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col] = df_cleaned[col].astype(str)
/var/folders/_d/pn647pfx16x6j369mlcqzzn40000gn/T/ipykernel_27942/4026449429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

NameError: name 'endpoint' is not defined